## 1. Install Dependencies

In [ ]:
# Install required packages
# These cover all dependencies needed by model.py, dataset.py, and training

# Fix protobuf version conflict first
!pip uninstall -y protobuf
!pip install -q protobuf==3.20.3

# Install transformers and dependencies
!pip install -q transformers>=4.35.0 accelerate sentencepiece

# Install torch-geometric
!pip install -q torch-geometric torch-scatter torch-sparse --no-deps
!pip install -q huggingface-hub tqdm

# Note: torch, numpy, pandas, scikit-learn are pre-installed on Kaggle
print("✓ All dependencies installed")

## 2. Setup and Imports

In [ ]:
import os
import sys
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm
from pathlib import Path
import json
import gc

# Check environment
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    
# Check if running on Kaggle
IS_KAGGLE = os.path.exists('/kaggle')
print(f"Running on Kaggle: {IS_KAGGLE}")

## 3. Configuration (Kaggle-Optimized)

In [ ]:
# Data paths - UPDATE THESE if using different dataset name
if IS_KAGGLE:
    DATA_CONFIG = {
        'train_jsonl': '/kaggle/input/cora-gwm-data/cora_train_node_data.jsonl',
        'test_jsonl': '/kaggle/input/cora-gwm-data/cora_test_node_data.jsonl',
        'embedding_path': '/kaggle/input/cora-gwm-data/multi_hop_graph_embedding.pt',
    }
else:
    DATA_CONFIG = {
        'train_jsonl': '../data/cora/cora_train_node_data.jsonl',
        'test_jsonl': '../data/cora/cora_test_node_data.jsonl',
        'embedding_path': '../data/cora/multi_hop_graph_embedding.pt',
    }

# Model configuration - OPTION 1: 8B with 8-bit quantization (RECOMMENDED)
MODEL_CONFIG = {
    'llama_model': 'meta-llama/Meta-Llama-3-8B-Instruct',  # 8B model with quantization
    'graph_embedding_dim': 2048,
    'projector_hidden_dim': 2048,  # Reduced to save memory
    'num_hops': 5,
    'use_8bit': True,  # Enable 8-bit quantization
}

# OPTION 2: Use 3B model without quantization (comment out above, use this)
# MODEL_CONFIG = {
#     'llama_model': 'meta-llama/Llama-3.2-3B-Instruct',
#     'graph_embedding_dim': 2048,
#     'projector_hidden_dim': 2048,
#     'num_hops': 5,
#     'use_8bit': False,
# }

# Training configuration - OPTIMIZED FOR KAGGLE
TRAINING_CONFIG = {
    'batch_size': 2,  # Small batch for 8B model
    'gradient_accumulation_steps': 16,  # Effective batch = 32
    'learning_rate': 2e-4,
    'num_epochs': 5,  # Reduced for 12-hour limit
    'warmup_steps': 50,  # Reduced warmup
    'num_workers': 2,  # Reduced workers
    'output_dir': '/kaggle/working/checkpoints' if IS_KAGGLE else './checkpoints',
    'save_every': 1,
    'use_fp16': True,  # Mixed precision training
}

device = 'cuda' if torch.cuda.is_available() else 'cpu'

print("\n" + "="*70)
print(" "*20 + "KAGGLE-OPTIMIZED CONFIGURATION")
print("="*70)
print("\nData:")
for k, v in DATA_CONFIG.items():
    print(f"  {k}: {v}")
print("\nModel (Memory-Optimized):")
for k, v in MODEL_CONFIG.items():
    print(f"  {k}: {v}")
print("\nTraining (Kaggle-Optimized):")
for k, v in TRAINING_CONFIG.items():
    print(f"  {k}: {v}")
print(f"\nDevice: {device}")
print(f"Effective batch size: {TRAINING_CONFIG['batch_size'] * TRAINING_CONFIG['gradient_accumulation_steps']}")
print("="*70)

## 4. Copy Model Files

Copy the model architecture files to working directory.

In [ ]:
# Clone GitHub repo and copy model files to working directory

if IS_KAGGLE:
    print("="*70)
    print("Cloning GitHub repository...")
    print("="*70)
    
    # Clone your GitHub repo (UPDATE with your repo URL)
    GITHUB_REPO = "https://github.com/HiIamPhuc/GWM.git"
    
    # Clone repo
    !git clone {GITHUB_REPO} /kaggle/temp/gwm_repo
    
    # Copy model files from repo to working directory
    repo_path = Path("/kaggle/temp/gwm_repo/GWM/gwm_e")
    
    if repo_path.exists():
        print(f"\n✓ Repository cloned successfully")
        print(f"Source: {repo_path}")
        
        # Copy files
        !cp {repo_path}/model.py /kaggle/working/
        !cp {repo_path}/dataset.py /kaggle/working/
        
        print("\n✓ Copied model.py")
        print("✓ Copied dataset.py")
    else:
        print(f"\n❌ Repository path not found: {repo_path}")
        print("Please check the repository structure")
else:
    print("Running locally - files should be in current directory")

# Verify files exist
required_files = ['model.py', 'dataset.py']
missing_files = [f for f in required_files if not os.path.exists(f)]

if missing_files:
    print(f"\n❌ Missing files: {missing_files}")
    raise FileNotFoundError(f"Required files not found: {missing_files}")
else:
    print(f"\n✓ All required files ready:")
    print(f"  - model.py")
    print(f"  - dataset.py")

## 5. Load Model with Quantization

In [ ]:
!hf auth login --token 

In [ ]:
from model import GWM_E

print("="*70)
print(" "*20 + "Loading GWM-E Model")
print("="*70)

# Initialize model
print(f"\nLoading {MODEL_CONFIG['llama_model']}...")
if MODEL_CONFIG.get('use_8bit'):
    print("Using 8-bit quantization to fit in 16GB GPU...")
print("This may take 2-3 minutes...\n")

model = GWM_E(
    llama_model_path=MODEL_CONFIG['llama_model'],
    graph_embedding_dim=MODEL_CONFIG['graph_embedding_dim'],
    projector_hidden_dim=MODEL_CONFIG['projector_hidden_dim'],
    num_hops=MODEL_CONFIG['num_hops'],
    freeze_llm=True,
    use_8bit=MODEL_CONFIG.get('use_8bit', False),  # Pass 8-bit flag
)

print("✓ Model loaded successfully!")

# Calculate parameters
llm_params = sum(p.numel() for p in model.llm.parameters()) / 1e9
projector_params = sum(p.numel() for p in model.projector.parameters()) / 1e6
trainable_params = sum(p.numel() for p in model.projector.parameters() if p.requires_grad) / 1e6

print(f"\nModel Statistics:")
print(f"  LLaMA parameters: {llm_params:.2f}B (frozen)")
print(f"  Projector parameters: {projector_params:.2f}M (trainable)")
print(f"  Total trainable: {trainable_params:.2f}M")

# Check GPU memory
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    allocated = torch.cuda.memory_allocated(0) / 1e9
    reserved = torch.cuda.memory_reserved(0) / 1e9
    print(f"\nGPU Memory:")
    print(f"  Allocated: {allocated:.2f} GB")
    print(f"  Reserved: {reserved:.2f} GB")
    print(f"  Available: {torch.cuda.get_device_properties(0).total_memory / 1e9 - reserved:.2f} GB")

## 6. Load Datasets

In [ ]:
from dataset import create_dataloaders

print("="*70)
print(" "*20 + "Loading Datasets")
print("="*70)

train_loader, test_loader = create_dataloaders(
    train_jsonl=DATA_CONFIG['train_jsonl'],
    test_jsonl=DATA_CONFIG['test_jsonl'],
    embedding_path=DATA_CONFIG['embedding_path'],
    tokenizer=model.tokenizer,
    batch_size=TRAINING_CONFIG['batch_size'],
    num_workers=TRAINING_CONFIG['num_workers'],
    num_hops=MODEL_CONFIG['num_hops'],
)

print(f"\n✓ Datasets loaded successfully!")
print(f"  Training samples: {len(train_loader.dataset):,}")
print(f"  Test samples: {len(test_loader.dataset):,}")
print(f"  Training batches: {len(train_loader):,}")
print(f"  Test batches: {len(test_loader):,}")

## 7. Setup Training

In [ ]:
print("="*70)
print(" "*20 + "Setting up Training")
print("="*70)

# Create output directory
output_dir = Path(TRAINING_CONFIG['output_dir'])
output_dir.mkdir(parents=True, exist_ok=True)

# Save config
config_path = output_dir / "training_config.json"
all_config = {**DATA_CONFIG, **MODEL_CONFIG, **TRAINING_CONFIG}
with open(config_path, 'w') as f:
    json.dump(all_config, f, indent=2)
print(f"✓ Saved config to: {config_path}")

# Optimizer
optimizer = torch.optim.AdamW(
    model.projector.parameters(),
    lr=TRAINING_CONFIG['learning_rate'],
    weight_decay=0.01,
)
print(f"✓ Optimizer: AdamW (lr={TRAINING_CONFIG['learning_rate']})")

# Scheduler
total_steps = len(train_loader) * TRAINING_CONFIG['num_epochs'] // TRAINING_CONFIG['gradient_accumulation_steps']
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=TRAINING_CONFIG['warmup_steps'],
    num_training_steps=total_steps,
)
print(f"✓ Scheduler: Linear warmup + decay")
print(f"  Total steps: {total_steps:,}")
print(f"  Warmup steps: {TRAINING_CONFIG['warmup_steps']:,}")

# Mixed precision (optional)
scaler = None
if TRAINING_CONFIG.get('use_fp16', False) and torch.cuda.is_available():
    scaler = torch.cuda.amp.GradScaler()
    print("✓ Mixed precision (FP16) enabled")

print("\n✓ Training setup complete!")

## 8. Training Functions

In [ ]:
def train_epoch(model, train_loader, optimizer, scheduler, epoch, gradient_accumulation_steps, device, scaler=None):
    model.train()
    total_loss = 0
    num_batches = 0
    
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch}")
    
    for batch_idx, batch in enumerate(progress_bar):
        multi_hop_embedding = batch['multi_hop_embedding'].to(device)
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        # Mixed precision forward pass
        if scaler is not None:
            with torch.cuda.amp.autocast():
                logits, loss = model(
                    multi_hop_embeddings=multi_hop_embedding,
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    labels=labels,
                )
            loss = loss / gradient_accumulation_steps
            scaler.scale(loss).backward()
        else:
            logits, loss = model(
                multi_hop_embeddings=multi_hop_embedding,
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels,
            )
            loss = loss / gradient_accumulation_steps
            loss.backward()
        
        # Update weights
        if (batch_idx + 1) % gradient_accumulation_steps == 0:
            if scaler is not None:
                scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.projector.parameters(), max_norm=1.0)
            
            if scaler is not None:
                scaler.step(optimizer)
                scaler.update()
            else:
                optimizer.step()
            
            scheduler.step()
            optimizer.zero_grad()
        
        total_loss += loss.item() * gradient_accumulation_steps
        num_batches += 1
        
        progress_bar.set_postfix({
            'loss': f"{loss.item() * gradient_accumulation_steps:.4f}",
            'lr': f"{scheduler.get_last_lr()[0]:.2e}"
        })
    
    return total_loss / num_batches


def evaluate(model, test_loader, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Evaluating"):
            multi_hop_embedding = batch['multi_hop_embedding'].to(device)
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            logits, loss = model(
                multi_hop_embeddings=multi_hop_embedding,
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels,
            )
            
            total_loss += loss.item()
            
            mask = labels != -100
            predictions = logits.argmax(dim=-1)
            correct += ((predictions == labels) & mask).sum().item()
            total += mask.sum().item()
    
    avg_loss = total_loss / len(test_loader)
    accuracy = correct / total if total > 0 else 0
    
    return avg_loss, accuracy


print("✓ Training functions defined")

## 9. Training Loop

In [ ]:
print("="*70)
print(" "*20 + "STARTING TRAINING (KAGGLE-OPTIMIZED)")
print("="*70)

best_accuracy = 0
training_history = []

for epoch in range(1, TRAINING_CONFIG['num_epochs'] + 1):
    print(f"\n{'='*70}")
    print(f"Epoch {epoch}/{TRAINING_CONFIG['num_epochs']}")
    print(f"{'='*70}")
    
    # Train
    train_loss = train_epoch(
        model=model,
        train_loader=train_loader,
        optimizer=optimizer,
        scheduler=scheduler,
        epoch=epoch,
        gradient_accumulation_steps=TRAINING_CONFIG['gradient_accumulation_steps'],
        device=device,
        scaler=scaler,
    )
    
    # Evaluate
    test_loss, test_accuracy = evaluate(
        model=model,
        test_loader=test_loader,
        device=device,
    )
    
    # Log results
    print(f"\n{'='*70}")
    print(f"Epoch {epoch} Results:")
    print(f"{'='*70}")
    print(f"  Train Loss:     {train_loss:.4f}")
    print(f"  Test Loss:      {test_loss:.4f}")
    print(f"  Test Accuracy:  {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
    
    # Save history
    training_history.append({
        'epoch': epoch,
        'train_loss': train_loss,
        'test_loss': test_loss,
        'test_accuracy': test_accuracy,
    })
    
    # Save checkpoint
    checkpoint_path = output_dir / f"projector_epoch_{epoch}.pt"
    model.save_projector(str(checkpoint_path))
    print(f"  ✓ Saved: {checkpoint_path.name}")
    
    if test_accuracy > best_accuracy:
        best_accuracy = test_accuracy
        best_path = output_dir / "projector_best.pt"
        model.save_projector(str(best_path))
        print(f"  ⭐ New best: {best_accuracy:.4f} ({best_accuracy*100:.2f}%)")
    
    # Memory cleanup
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        gc.collect()

# Save history
history_path = output_dir / "training_history.json"
with open(history_path, 'w') as f:
    json.dump(training_history, f, indent=2)

print("\n" + "="*70)
print(" "*20 + "✅ TRAINING COMPLETED!")
print("="*70)
print(f"Best accuracy: {best_accuracy:.4f} ({best_accuracy*100:.2f}%)")
print(f"Checkpoints: {output_dir}")
print("="*70)

## 10. Visualize Training Progress

In [ ]:
import matplotlib.pyplot as plt

# Extract metrics
epochs = [h['epoch'] for h in training_history]
train_losses = [h['train_loss'] for h in training_history]
test_losses = [h['test_loss'] for h in training_history]
test_accuracies = [h['test_accuracy'] for h in training_history]

# Create plots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Loss plot
ax1.plot(epochs, train_losses, 'b-o', label='Train Loss')
ax1.plot(epochs, test_losses, 'r-o', label='Test Loss')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training and Test Loss')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Accuracy plot
ax2.plot(epochs, [acc * 100 for acc in test_accuracies], 'g-o', label='Test Accuracy')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy (%)')
ax2.set_title('Test Accuracy')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / 'training_curves.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"✓ Saved training curves to: {output_dir / 'training_curves.png'}")

## 11. Summary and Download

In [ ]:
print("="*70)
print(" "*25 + "TRAINING SUMMARY")
print("="*70)

print(f"\n📊 Results:")
print(f"  Best Test Accuracy: {best_accuracy:.4f} ({best_accuracy*100:.2f}%)")
print(f"  Final Train Loss:   {training_history[-1]['train_loss']:.4f}")
print(f"  Final Test Loss:    {training_history[-1]['test_loss']:.4f}")

print(f"\n💾 Saved Files:")
for file in sorted(output_dir.glob("*")):
    size = os.path.getsize(file) / (1024**2)
    print(f"  • {file.name} ({size:.1f} MB)")

if IS_KAGGLE:
    print(f"\n📥 Download from Kaggle:")
    print(f"  1. Go to 'Output' tab (right sidebar)")
    print(f"  2. Download 'checkpoints/' folder")
    print(f"  3. Key file: projector_best.pt (~{os.path.getsize(output_dir / 'projector_best.pt') / (1024**2):.1f} MB)")

print(f"\n🚀 Next Steps:")
print(f"  1. Download projector_best.pt")
print(f"  2. Run inference to evaluate on test set")
print(f"  3. Use projector with any LLaMA model")

print("\n" + "="*70)

## Optional: Test a Single Prediction

Quick test to see model predictions.

In [ ]:
# Get a sample from test set
test_sample = test_loader.dataset[0]
test_conv = test_loader.dataset.conversations[0]

print("Testing model on a single example...\n")
print("="*70)
print("Sample Input:")
print("="*70)
for turn in test_conv['conversations']:
    role = turn['from']
    message = turn['value'][:200] + "..." if len(turn['value']) > 200 else turn['value']
    print(f"{role.upper()}: {message}\n")

# Prepare inputs
multi_hop_emb = test_sample['multi_hop_embedding'].unsqueeze(0).to(device)
input_ids = test_sample['input_ids'].unsqueeze(0).to(device)
attention_mask = test_sample['attention_mask'].unsqueeze(0).to(device)

# Generate prediction
model.eval()
with torch.no_grad():
    outputs = model.generate(
        multi_hop_embeddings=multi_hop_emb,
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=50,
        temperature=0.1,
    )

# Decode
input_length = input_ids.shape[1]
generated = model.tokenizer.decode(outputs[0][input_length:], skip_special_tokens=True)

print("="*70)
print("Model Prediction:")
print("="*70)
print(f"ASSISTANT: {generated}\n")
print("="*70)